In [69]:
import json
import os
import glob
import pickle

In [70]:
data_root = 'dd/LAED/2019-12-04T23-43-57-dailydial-skip.py/'

In [71]:
greedy_outf = glob.glob(data_root+'/*greedy.txt.txt')[0]
print(greedy_outf)

dd/LAED/2019-12-04T23-43-57-dailydial-skip.py/2019-12-04T23-44-05-test-greedy.txt.txt


In [72]:
tag = 'laed-skip'

In [73]:
utterances = []
# Process the individual predictions from LAED
with open(greedy_outf) as f:
    dat = {}
    for line in f:
        if line.startswith("==== REPORT===="):
            break
        if len(line.strip()) == 0:
            if len(dat) > 0:
                utterances.append(dat)
            dat = {}
            continue
        
        k, v = line.split(': ', 1)
        k = k.strip()
        v = v.strip()
        dat[k] = v

laed_predictions = []
dial = []
for entry in utterances:
    if entry['Source'] == '<d>':
        if len(dial) > 0:
            laed_predictions.append(dial)
        dial = []
        
    dial.append(entry)

# THE LAST ONE    
laed_predictions.append(dial)

In [74]:
vlgan_predictions = []
with open('dd/VLGAN/dd_predictions_tmp.json') as vlganf:
    for line in vlganf:
        po = json.loads(line)
        vlgan_predictions.append(po)

In [75]:
len(vlgan_predictions)

1000

In [76]:
for i in range(len(vlgan_predictions)):
    laed_sample, vlgan_sample = laed_predictions[i], vlgan_predictions[i]
    if len(laed_sample) < 12:
        assert (len(laed_sample)) == len(vlgan_sample)
    else:
        assert len(vlgan_sample) == 11 and len(laed_sample) > 11
    
    for t in range(min(12, len(vlgan_sample))):
        _t = str(t)
        vlgan_predictions[i][_t][f'{tag}'] = laed_sample[t]['Predict']
        vlgan_predictions[i][_t][f'{tag}.Target'] = laed_sample[t]['Target']

In [77]:
vlgan_predictions[-988]

{'0': {'user-x': 0,
  'gold': "I'm sorry I'm so late . I had a really bad day .",
  'laed-skip': "television? i 'm a foreigner. i haven' t seen a lot of gas lately.",
  'laed-skip.Target': "i 'm sorry i' m so late. i had a really bad day."},
 '1': {'user-x': 1,
  'gold': "It's ten after six.We ' re late.But dinner is at six thirty .",
  'resgen': ['you re welcome to the . . .'],
  'laed-skip': 'did you panic it?',
  'laed-skip.Target': "it 's ten after six. we' re late. but dinner is at six thirty."},
 '2': {'user-x': 0,
  'gold': "I know . I know . I'm really sorry . I lost my bag .",
  'resgen': ['oh , that s too .'],
  'laed-skip': 'white.',
  'laed-skip.Target': "i know. i know. i 'm really sorry. i lost my bag."},
 '3': {'user-x': 1,
  'gold': "I'll call the lost and found office .",
  'resgen': ['you are too cold .'],
  'laed-skip': "appropriate. you should go to the gym with us. i think you 're going to be there.",
  'laed-skip.Target': "i 'll call the lost and found office."},


In [78]:
laed_sample

[{'Source': '<d>',
  'Target': 'what a nice day!',
  'Predict': '325, are you going to have a nice trip?'},
 {'Source': '<d><t>what a nice day!',
  'Target': 'yes. how about going out and enjoying the sunshine on the grass?',
  'Predict': 'what do you think of the traffic?'},
 {'Source': '<d><t>what a nice day!<t>yes. how about going out and enjoying the sunshine on the grass?',
  'Target': "great, let 's go!",
  'Predict': 'iq, i want to be a new car.'},
 {'Source': "<d><t>what a nice day!<t>yes. how about going out and enjoying the sunshine on the grass?<t>great, let 's go!",
  'Target': 'hey, darling, i think i might have a little <unk> from being in the sun all day. i was so relaxed. it felt as if i were in another world.',
  'Predict': 'figured!'},
 {'Source': "<d><t>what a nice day!<t>yes. how about going out and enjoying the sunshine on the grass?<t>great, let 's go!<t>hey, darling, i think i might have a little <unk> from being in the sun all day. i was so relaxed. it felt as i